<a href="https://colab.research.google.com/github/egs1sos/IS-4487/blob/main/lab_07_data_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> # ⚠️ **IMPORTANT: READ BEFORE STARTING THIS LAB**
>
> ### Throughout this lab, you will see **🔧 Try It Yourself** sections and a final 🔧 **Reflection** section
>
> ✅ You are expected to:
> - Complete each **"🔧 Try It Yourself”** section by writing and running your own code or answering the prompted questions in a markdown or python cell below the section.
> - Answer the **Reflection** section at the end of the lab in your own words. This is your opportunity to summarize what you learned and connect the concepts.

>
>
> 🔧 Look for the **wrench emoji** 🔧 — it highlights where you're expected to take action!
>
> ### These sections are **graded** and are **not optional**. Skipping them will impact your lab score.
>
> ---

# IS 4487 Lab 7: Data Transformation

## Outline

- Load and preview the cleaned Megatelco dataset  
- Engineer new columns from existing data  
- Simplify or group variable values  
- Use `.map()`, `.apply()`, and `pd.cut()` for transformations  
- Try your own transformation logic  

This lab continues from **Lab 6**, where we cleaned the Megatelco dataset.  

Now, we will create new, more useful features for modeling and exploration.

<a href="https://colab.research.google.com/github/Stan-Pugsley/is_4487_base/blob/main/Labs/lab_07_data_transformation.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

If you're new to Colab: [Colab FAQ](https://research.google.com/colaboratory/faq.html)




## Megatelco Data Dictionary

 DEMOGRAPHIC VARIABLES:
 - College - has the customer attended some college (one, zero)
 - Income - annual income of customer
 - House - estimated price of the customer's home (if applicable)

 USAGE VARIABLES:
 - Data Overage Mb - Average number of megabytes that the customer used in excess of the plan limit (over last 12 months)
 - Data Leftover Mb - Average number of megabytes that the customer use was below the plan limit (over last 12 months)
 - Data Mb Used - Average number of megabytes used per month (over last 12 months)
 - Text Message Count - Average number of texts per month (over last 12 months)
 - Over 15 Minute Calls Per Month - Average number of calls over 15 minutes in duration per month (over last 12 months)
 - Average Call Duration- Average call duration (over last 12 months)

PHONE VARIABLES:
 - Operating System - Current operating system of phone
 - Handset Price - Retail price of the phone used by the customer

ATTITUDINAL VARIABLES:
 - Reported Satisfaction - Survey response to "How satisfied are you with your current phone plan?" (high, med, low)
 - Reported Usage Level - Survey response to "How much do your use your phone?" (high, med, low)
 - Considering Change of Plan - Survey response to "Are you currently planning to change companies when your contract expires?" (high, med, low)

OTHER VARIABLES
 - Leave - Did this customer churn with the last contract expiration? (LEAVE, STAY)
 - ID - Customer identifier

# Part 1: Data Cleaning steps from Lab 6

In this part of the lab, we will load the cleaning steps previously done in lab 6

- Load the Megatelco dataset
- Clean column names
- Fix data types
- Handle missing values
- Remove duplicate records
- Review for outliers


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

url = "https://raw.githubusercontent.com/Stan-Pugsley/is_4487_base/812e9f15c357a5657a2795631fcaa9d9363cb417/DataSets/megatelco_leave_survey_data_cleaning_v2.csv"
df = pd.read_csv(url)

df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15016 entries, 0 to 15015
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   college                      15016 non-null  object 
 1   income                       15006 non-null  float64
 2   data_overage_mb              15016 non-null  int64  
 3   data_leftover_mb             14916 non-null  float64
 4   data_mb_used                 14916 non-null  float64
 5   text_message_count           15016 non-null  int64  
 6   house                        15016 non-null  int64  
 7   handset_price                14916 non-null  float64
 8   over_15mins_calls_per_month  15013 non-null  float64
 9   average_call_duration        14916 non-null  float64
 10  reported_satisfaction        15016 non-null  object 
 11  reported_usage_level         15016 non-null  object 
 12  considering_change_of_plan   14201 non-null  object 
 13  leave           

In [ ]:
# Standardize column names
# - Remove leading/trailing whitespace
# - Convert to lowercase
# - Replace spaces with underscores
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Show info about data types and non-null values
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15016 entries, 0 to 15015
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   college                      15016 non-null  object 
 1   income                       15006 non-null  float64
 2   data_overage_mb              15016 non-null  int64  
 3   data_leftover_mb             14916 non-null  float64
 4   data_mb_used                 14916 non-null  float64
 5   text_message_count           15016 non-null  int64  
 6   house                        15016 non-null  int64  
 7   handset_price                14916 non-null  float64
 8   over_15mins_calls_per_month  15013 non-null  float64
 9   average_call_duration        14916 non-null  float64
 10  reported_satisfaction        15016 non-null  object 
 11  reported_usage_level         15016 non-null  object 
 12  considering_change_of_plan   14201 non-null  object 
 13  leave           

In [ ]:
# Check original data types
print("Original dtypes:\n", df.dtypes)

# Convert yes/no columns to binary categories
df['considering_change_of_plan'] = df['considering_change_of_plan'].map({'yes': 1, 'no': 0}).astype('category')
df['leave'] = df['leave'].map({'yes': 1, 'no': 0}).astype('category')  # newly added

# Convert categorical text columns to 'category' dtype
df['college'] = df['college'].astype('category')
df['house'] = df['house'].astype('category')
df['reported_satisfaction'] = df['reported_satisfaction'].astype('category')
df['reported_usage_level'] = df['reported_usage_level'].astype('category')  # newly added
df['operating_system'] = df['operating_system'].astype('category')

# Convert numeric-looking columns from object to float
df['income'] = pd.to_numeric(df['income'], errors='coerce')
df['data_overage_mb'] = pd.to_numeric(df['data_overage_mb'], errors='coerce')
df['data_leftover_mb'] = pd.to_numeric(df['data_leftover_mb'], errors='coerce')
df['data_mb_used'] = pd.to_numeric(df['data_mb_used'], errors='coerce')
df['text_message_count'] = pd.to_numeric(df['text_message_count'], errors='coerce')
df['handset_price'] = pd.to_numeric(df['handset_price'], errors='coerce')  # newly added
df['over_15mins_calls_per_month'] = pd.to_numeric(df['over_15mins_calls_per_month'], errors='coerce')
df['average_call_duration'] = pd.to_numeric(df['average_call_duration'], errors='coerce')

# Check updated data types
print("\nUpdated dtypes:\n", df.dtypes)


Original dtypes:
 college                         object
income                         float64
data_overage_mb                  int64
data_leftover_mb               float64
data_mb_used                   float64
text_message_count               int64
house                            int64
handset_price                  float64
over_15mins_calls_per_month    float64
average_call_duration          float64
reported_satisfaction           object
reported_usage_level            object
considering_change_of_plan      object
leave                           object
id                               int64
operating_system                object
dtype: object

Updated dtypes:
 college                        category
income                          float64
data_overage_mb                   int64
data_leftover_mb                float64
data_mb_used                    float64
text_message_count                int64
house                          category
handset_price                   float64
over_1

In [ ]:
# View missing value counts
print("Missing values per column:\n", df.isnull().sum())

# Fill 'handset_price' with median value
df['handset_price'] = df['handset_price'].fillna(df['handset_price'].median())

# Drop rows with missing income (assuming it's a small number of rows)
df = df.dropna(subset=['income'])

# Fill missing data_leftover_mb with 0 (if no leftover means truly zero)
df['data_leftover_mb'] = df['data_leftover_mb'].fillna(0)

# Fill missing call duration and data usage with their respective medians
df['average_call_duration'] = df['average_call_duration'].fillna(df['average_call_duration'].median())
df['data_mb_used'] = df['data_mb_used'].fillna(df['data_mb_used'].median())

# Check updated missing values
print("\nMissing values after handling:\n", df.isnull().sum())

Missing values per column:
 college                            0
income                            10
data_overage_mb                    0
data_leftover_mb                 100
data_mb_used                     100
text_message_count                 0
house                              0
handset_price                    100
over_15mins_calls_per_month        3
average_call_duration            100
reported_satisfaction              0
reported_usage_level               0
considering_change_of_plan       815
leave                          15016
id                                 0
operating_system                   0
dtype: int64

Missing values after handling:
 college                            0
income                             0
data_overage_mb                    0
data_leftover_mb                   0
data_mb_used                       0
text_message_count                 0
house                              0
handset_price                      0
over_15mins_calls_per_month        3
a

In [ ]:
# Check for exact duplicates
print(f"Number of duplicate rows: {df.duplicated().sum()}")

# Remove duplicate rows, keeping the first instance
df = df.drop_duplicates()

# Confirm number of rows remaining
print(f"Remaining rows after removing duplicates: {len(df)}")

Number of duplicate rows: 17
Remaining rows after removing duplicates: 14989


In [ ]:
# Calculate 1st and 99th percentiles for income
income_min, income_max = df['income'].quantile([0.01, 0.99])

# Use .loc to avoid SettingWithCopyWarning and ensure assignment modifies the original DataFrame
df.loc[:, 'income'] = df['income'].clip(lower=income_min, upper=income_max)

# Clip 'data_mb_used' to within 1st and 99th percentiles
usage_min, usage_max = df['data_mb_used'].quantile([0.01, 0.99])
df.loc[:, 'data_mb_used'] = df['data_mb_used'].clip(lower=usage_min, upper=usage_max)

# Clip 'average_call_duration' to reduce the effect of extreme outliers
call_min, call_max = df['average_call_duration'].quantile([0.01, 0.99])
df.loc[:, 'average_call_duration'] = df['average_call_duration'].clip(lower=call_min, upper=call_max)


## Part 2: Creating New Features

A major part of data preparation is **feature engineering** — creating new columns from raw data to capture useful patterns.

In this section, we will try three common methods:

1. `.map()` — useful for simplifying categories (e.g., satisfaction levels)
2. `.apply()` — allows flexible custom logic (e.g., flagging high usage)
3. `pd.cut()` or `pd.qcut()` — groups numeric values into bins or quantiles

These new features help models learn better and make reports easier to interpret.

Things to think about:
- Are any categories too specific or inconsistent?
- Can you create groups or flags to highlight important traits?
- Would a simplified version of a column help with modeling or visualization?

In [ ]:
# Create a total data usage variable (used + leftover)
df['total_data_mb'] = df['data_mb_used'] + df['data_leftover_mb']

# Create a ratio of overage to used data
df['overage_ratio'] = df['data_overage_mb'] / (df['data_mb_used'] + 1)  # add 1 to avoid divide-by-zero

# Create a binary flag for high texters (over 500 texts)
df['high_texter'] = (df['text_message_count'] > 500).astype(int)

# Preview new columns
df[['total_data_mb', 'overage_ratio', 'high_texter']].head()


,total_data_mb,overage_ratio,high_texter
0,6605.0,0.010596,0
1,6044.0,0.011113,0
2,1482.0,0.040459,0
3,3027.0,0.000000,0
4,1794.0,0.000000,0


### 🔧 Try It Yourself – Part 2

1. Create a variable called `call_volume` by multiplying `over_15mins_calls_per_month` by `average_call_duration`
2. Create a binary flag `high_data_user` for users where `data_mb_used` is above the median
3. Use `.head()` to check your new columns



In [ ]:
# 🔧 Add code here
df['call_volume'] = df['over_15mins_calls_per_month'] * df['average_call_duration']
df['high_data_user'] = (df['data_mb_used'] > df['data_mb_used'].median()).astype(int)
df[['call_volume', 'high_data_user']].head()

,call_volume,high_data_user
0,40.0,1
1,25.0,1
2,24.0,0
3,0.0,0
4,0.0,0


## Part 3: Binning Continous Variables

Binning is the process of grouping numeric variables into categories (e.g., "low", "medium", "high").

### Why We Bin:
- Helps reduce the impact of outliers
- Allows us to use numeric values in models that prefer categories
- Simplifies interpretation and visualization

### Things to think about:
- Would grouping values make patterns more visible?
- Do we want equal-sized groups or logical cutoffs?
- Is the variable skewed?

**Tools:**  
- `pd.qcut()` for quantile-based bins (equal frequency)  
- `pd.cut()` for equal-width or custom bins


In [ ]:
# Bin income into 3 groups (quantiles): Low, Medium, High
df['income_group'] = pd.qcut(df['income'], q=3, labels=['Low', 'Medium', 'High'])

# Bin average call duration into quartiles (labels as integers)
df['call_duration_group'] = pd.qcut(df['average_call_duration'], q=4, labels=False)

# Preview new groupings
df[['income', 'income_group', 'average_call_duration', 'call_duration_group']].head()

,income,income_group,average_call_duration,call_duration_group
0,403137.0,High,8.0,1
1,129700.0,Low,5.0,0
2,69741.0,Low,8.0,1
3,377572.0,High,5.0,0
4,382080.0,High,14.0,2


### 🔧 Try It Yourself – Part 3

1. Use `pd.cut()` to group `data_mb_used` into 3 labeled bins: "Light", "Moderate", "Heavy"
2. Use `pd.qcut()` on `text_message_count` to split into 4 equal-sized groups
3. Print `.value_counts()` on each new column to see how values are distributed

In [ ]:
# 🔧 Add code here
pd.cut(df['data_mb_used'], bins=3, labels=['Light', 'Moderate', 'Heavy'])
pd.qcut(df['text_message_count'], q=4)
print(df['data_mb_used'].value_counts())

data_mb_used
471.00     152
7929.12    150
4221.00    102
5656.00      8
5946.00      8
          ... 
2485.00      1
1472.00      1
6247.00      1
1750.00      1
5220.00      1
Name: count, Length: 6378, dtype: int64


## Part 4: Scaling Numeric Variables

Scaling transforms values to a common range (often 0–1), which helps many machine learning models perform better.

### When to Scale:
- When features have very different ranges (e.g., income vs. call duration)
- When using distance-based models (e.g., KNN, SVM)
- When comparing magnitudes across features

### Common Methods:
- `MinMaxScaler`: scales to 0–1 range
- `StandardScaler`: centers data around 0 with unit variance

### Things to think about:
- Are features on different scales?
- Does my algorithm care about magnitude?

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Choose columns to scale
cols_to_scale = ['income', 'data_mb_used', 'average_call_duration']

# Initialize and apply scaler
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[cols_to_scale])

# Add scaled columns back to df
df['income_scaled'] = df_scaled[:, 0]
df['data_mb_used_scaled'] = df_scaled[:, 1]
df['avg_call_dur_scaled'] = df_scaled[:, 2]

# Preview
df[['income_scaled', 'data_mb_used_scaled', 'avg_call_dur_scaled']].head()

,income_scaled,data_mb_used_scaled,avg_call_dur_scaled
0,0.933230,0.822459,0.388889
1,0.198757,0.745094,0.222222
2,0.037703,0.135557,0.388889
3,0.864560,0.339764,0.222222
4,0.876669,0.177391,0.722222


### 🔧 Try It Yourself – – Part 4

1. Scale the `handset_price` and `over_15mins_calls_per_month` columns using `MinMaxScaler`
2. Add the scaled values back to the dataframe with suffix `_scaled`
3. Use `.describe()` to compare original vs. scaled versions and make a comment on what you observe


In [ ]:
# 🔧 Add code here
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[['handset_price', 'over_15mins_calls_per_month']])
df['handset_price_scaled'] = df_scaled[:, 0]
df['over_15mins_calls_per_month_scaled'] = df_scaled[:, 1]
df.describe()

,income,data_overage_mb,data_leftover_mb,data_mb_used,text_message_count,handset_price,over_15mins_calls_per_month,average_call_duration,id,total_data_mb,overage_ratio,high_texter,call_volume,high_data_user,call_duration_group,income_scaled,data_mb_used_scaled,avg_call_dur_scaled,handset_price_scaled,over_15mins_calls_per_month_scaled
count,14989.000000,14989.000000,14989.000000,14989.000000,14989.000000,14989.000000,14986.000000,14989.000000,14989.000000,14989.000000,14989.000000,14989.000000,14986.000000,14989.000000,14989.000000,14989.000000,14989.000000,14989.000000,14989.000000,14986.000000
mean,241993.010341,153.526586,37.261392,4201.098806,135.951431,795.006938,10.570266,9.732337,11859.840883,4238.360197,0.061042,0.000067,107.215801,0.496497,1.418574,0.500385,0.500139,0.485130,0.007947,0.302008
std,109519.502519,113.009107,28.118539,2195.602095,62.952477,1235.894864,8.401910,5.178558,6813.059020,2195.891603,0.087968,0.008168,106.621054,0.500004,1.091182,0.294178,0.294391,0.287698,0.009871,0.240055
min,55704.640000,0.000000,0.000000,471.000000,52.000000,-200.000000,0.000000,1.000000,2.000000,471.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,147806.000000,54.000000,12.000000,2304.000000,93.000000,499.000000,3.000000,5.000000,6137.000000,2336.000000,0.012136,0.000000,19.000000,0.000000,0.000000,0.247391,0.245772,0.222222,0.005583,0.085714
50%,241656.000000,151.000000,34.000000,4221.000000,135.000000,777.000000,9.000000,10.000000,11762.000000,4258.000000,0.034975,0.000000,75.000000,0.000000,1.000000,0.499479,0.502808,0.500000,0.007804,0.257143
75%,336443.000000,242.000000,62.000000,6063.000000,178.000000,1062.000000,17.000000,14.000000,17398.000000,6099.000000,0.068481,0.000000,168.000000,1.000000,2.000000,0.754085,0.749787,0.722222,0.010080,0.485714
max,427994.960000,380.000000,89.000000,7929.120000,5000.000000,125000.000000,35.000000,19.000000,25354.000000,8018.120000,0.800847,1.000000,630.000000,1.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Add comment here
I noticed that the scaled values are on a range from 0 to 1, but have the same count of values.

## Part 5: Encoding Categorical Variables

Most machine learning models can't handle string categories directly—so we convert them into numbers using **encoding**.

### Types of Encoding:
- **One-hot encoding**: creates a binary column for each category (preferred for nominal variables)
- **Ordinal encoding**: assigns integers (use only for ordered categories)

### Things to consider:
- Is the variable nominal (e.g., OS type) or ordinal (e.g., satisfaction)?
- How many unique categories are there?
- Will one-hot encoding make the dataset too wide?

**Tool:** `pd.get_dummies()`

In [ ]:
# One-hot encode 'reported_usage_level'
df_encoded = pd.get_dummies(df, columns=['reported_usage_level'], prefix='usage')

# One-hot encode 'income_group'
df_encoded = pd.get_dummies(df_encoded, columns=['income_group'], prefix='income')

# Preview new columns
df_encoded.filter(like='usage_').head()

,usage_avg,usage_high,usage_low
0,False,False,True
1,False,False,True
2,False,False,True
3,False,False,True
4,False,False,True


### 🔧 Try It Yourself – Part 5

1. One-hot encode `reported_satisfaction` and `operating_system`
2. Print `.shape` of your dataframe before and after to observe any big changes
3. How many new columns were added?


In [ ]:
# 🔧 Add code here
df_encoded = pd.get_dummies(df_encoded, columns=['reported_satisfaction'], prefix='satisfaction')
df_encoded = pd.get_dummies(df_encoded, columns=['operating_system'], prefix='os')
print(df_encoded.shape)

(14989, 33)


🔧 Add comment here: It looks like there are 33 columns now, with 2 new columns.

# 🔧 Part 6: Reflection (100 words or less per question)

1. Which transformation do you think had the biggest impact on preparing your data for modeling?
2. Are there any features you created that you think will be especially useful for predicting churn?

🔧 Add comment here:
1. I think the biggest transformation was the feature engineeering, because it is an entirely new column that lets us look for patterns in existing data. In this case, I found call volume to determine whether a user was a heavy user.
2. I think the new call volume column is useful in predicting churn, as heavy users are less likely to switch.